In [9]:
# #--- 초기 패키지 설치(아래 코드가 안될때도 실행) ---#

# # (240424) 코드 일부 업데이트되었습니다.

#!python -m pip install --upgrade pip
#!pip uninstall selenium -y
#!pip install selenium==4.25.0

#!pip uninstall webdriver-manager -y
#!pip install webdriver-manager -U

#!pip uninstall undetected_chromedriver -y
#!pip install undetected_chromedriver -U
#!pip install user_agents

#!pip install pyperclip

#!pip install chromedriver_autoinstaller

#--- 라이브러리 import ---#
import pandas as pd
# DataFrame을 컨트롤하는 라이브러리

import warnings, os, time, shutil, urllib, random
# warnings : 파이썬 경고 제어
# os : 경로 또는 파일 제어
# time : 시간 제어(일정시간 delay등)
# shutil : os의 폴더 제어(생성/삭제)
# urlib : 특정 url에서 이미지를 받을때 사용

warnings.filterwarnings(action='ignore')
# 파이썬에서 불필요한 warning은 안뜨게 함

from PIL import Image
# 파이썬에서 이미지를 다룰때 사용(png등을 불러들여서 리사이즈 저장 등)

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# Keys.ARROW_DOWN , Keys.ARROW_LEFT , Keys.ARROW_RIGHT, Keys.ARROW_UP , Keys.BACK_SPACE , Keys.CONTROL, Keys.ALT , Keys.DELETE , 
# Keys.ENTER , Keys.SHIFT, Keys.SPACE , Keys.TAB , Keys.EQUALS , Keys.ESCAPE, Keys.HOME , Keys.INSERT , PgUp Key,  Keys.PAGE_UP,
# Keys.PAGE_DOWN , Keys.F1 , Keys.F2 , Keys.F3 , Keys.F4, Keys.F5 , Keys.F6 , Keys.F7 , Keys.F8 , Keys.F9 , Keys.F10, Keys.F11 , Keys.F12
from selenium.webdriver import ActionChains

# 🟢 새로 추가된 import문들
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# 🟢 상수 정의 추가
WAIT_TIMEOUT = 10

# 🎯 검색할 도시들 설정 - 여기서 원하는 도시로 변경하세요!
CITIES_TO_SEARCH = ["방콕"]  # 기본 3개 (테스트용)
# CITIES_TO_SEARCH = ["방콕", "도쿄", "파리", "뉴욕", "런던"]  # 5개
# CITIES_TO_SEARCH = ["방콕", "도쿄"]  # 2개 (빠른 테스트)

# 🌏 도시별 대륙 및 국가 정보 매핑
CITY_INFO = {
    # 아시아
    "방콕": {"대륙": "아시아", "국가": "태국"},
    "도쿄": {"대륙": "아시아", "국가": "일본"},
    "오사카": {"대륙": "아시아", "국가": "일본"},
    "교토": {"대륙": "아시아", "국가": "일본"},
    "싱가포르": {"대륙": "아시아", "국가": "싱가포르"},
    "홍콩": {"대륙": "아시아", "국가": "홍콩"},
    "타이베이": {"대륙": "아시아", "국가": "대만"},
    "상하이": {"대륙": "아시아", "국가": "중국"},
    "베이징": {"대륙": "아시아", "국가": "중국"},
    "푸켓": {"대륙": "아시아", "국가": "태국"},
    "파타야": {"대륙": "아시아", "국가": "태국"},
    "치앙마이": {"대륙": "아시아", "국가": "태국"},
    "호치민": {"대륙": "아시아", "국가": "베트남"},
    "하노이": {"대륙": "아시아", "국가": "베트남"},
    "다낭": {"대륙": "아시아", "국가": "베트남"},
    "세부": {"대륙": "아시아", "국가": "필리핀"},
    "보라카이": {"대륙": "아시아", "국가": "필리핀"},
    "발리": {"대륙": "아시아", "국가": "인도네시아"},
    "자카르타": {"대륙": "아시아", "국가": "인도네시아"},
    "쿠알라룸푸르": {"대륙": "아시아", "국가": "말레이시아"},
    "코타키나발루": {"대륙": "아시아", "국가": "말레이시아"},
    "랑카위": {"대륙": "아시아", "국가": "말레이시아"},
    "페낭": {"대륙": "아시아", "국가": "말레이시아"},
    
    # 유럽
    "파리": {"대륙": "유럽", "국가": "프랑스"},
    "런던": {"대륙": "유럽", "국가": "영국"},
    "로마": {"대륙": "유럽", "국가": "이탈리아"},
    "밀라노": {"대륙": "유럽", "국가": "이탈리아"},
    "베니스": {"대륙": "유럽", "국가": "이탈리아"},
    "바르셀로나": {"대륙": "유럽", "국가": "스페인"},
    "마드리드": {"대륙": "유럽", "국가": "스페인"},
    "암스테르담": {"대륙": "유럽", "국가": "네덜란드"},
    "베를린": {"대륙": "유럽", "국가": "독일"},
    "뮌헨": {"대륙": "유럽", "국가": "독일"},
    "프라하": {"대륙": "유럽", "국가": "체코"},
    "비엔나": {"대륙": "유럽", "국가": "오스트리아"},
    "취리히": {"대륙": "유럽", "국가": "스위스"},
    "제네바": {"대륙": "유럽", "국가": "스위스"},
    "스톡홀름": {"대륙": "유럽", "국가": "스웨덴"},
    "코펜하겐": {"대륙": "유럽", "국가": "덴마크"},
    "헬싱키": {"대륙": "유럽", "국가": "핀란드"},
    "모스크바": {"대륙": "유럽", "국가": "러시아"},
    "상트페테르부르크": {"대륙": "유럽", "국가": "러시아"},
    "아테네": {"대륙": "유럽", "국가": "그리스"},
    "리스본": {"대륙": "유럽", "국가": "포르투갈"},
    "부다페스트": {"대륙": "유럽", "국가": "헝가리"},
    "바르샤바": {"대륙": "유럽", "국가": "폴란드"},
    
    # 북미
    "뉴욕": {"대륙": "북미", "국가": "미국"},
    "로스앤젤레스": {"대륙": "북미", "국가": "미국"},
    "라스베이거스": {"대륙": "북미", "국가": "미국"},
    "샌프란시스코": {"대륙": "북미", "국가": "미국"},
    "시카고": {"대륙": "북미", "국가": "미국"},
    "보스턴": {"대륙": "북미", "국가": "미국"},
    "마이애미": {"대륙": "북미", "국가": "미국"},
    "시애틀": {"대륙": "북미", "국가": "미국"},
    "하와이": {"대륙": "북미", "국가": "미국"},
    "밴쿠버": {"대륙": "북미", "국가": "캐나다"},
    "토론토": {"대륙": "북미", "국가": "캐나다"},
    "몬트리올": {"대륙": "북미", "국가": "캐나다"},
    "멕시코시티": {"대륙": "북미", "국가": "멕시코"},
    "칸쿤": {"대륙": "북미", "국가": "멕시코"},
    
    # 남미
    "리우데자네이루": {"대륙": "남미", "국가": "브라질"},
    "상파울루": {"대륙": "남미", "국가": "브라질"},
    "부에노스아이레스": {"대륙": "남미", "국가": "아르헨티나"},
    "리마": {"대륙": "남미", "국가": "페루"},
    "산티아고": {"대륙": "남미", "국가": "칠레"},
    "보고타": {"대륙": "남미", "국가": "콜롬비아"},
    "키토": {"대륙": "남미", "국가": "에콰도르"},
    
    # 오세아니아
    "시드니": {"대륙": "오세아니아", "국가": "호주"},
    "멜버른": {"대륙": "오세아니아", "국가": "호주"},
    "골드코스트": {"대륙": "오세아니아", "국가": "호주"},
    "퍼스": {"대륙": "오세아니아", "국가": "호주"},
    "애들레이드": {"대륙": "오세아니아", "국가": "호주"},
    "케언즈": {"대륙": "오세아니아", "국가": "호주"},
    "오클랜드": {"대륙": "오세아니아", "국가": "뉴질랜드"},
    "크라이스트처치": {"대륙": "오세아니아", "국가": "뉴질랜드"},
    "웰링턴": {"대륙": "오세아니아", "국가": "뉴질랜드"},
    
    # 중동
    "두바이": {"대륙": "중동", "국가": "UAE"},
    "아부다비": {"대륙": "중동", "국가": "UAE"},
    "도하": {"대륙": "중동", "국가": "카타르"},
    "이스탄불": {"대륙": "중동", "국가": "터키"},
    "텔아비브": {"대륙": "중동", "국가": "이스라엘"},
    "예루살렘": {"대륙": "중동", "국가": "이스라엘"},
    
    # 아프리카
    "카이로": {"대륙": "아프리카", "국가": "이집트"},
    "케이프타운": {"대륙": "아프리카", "국가": "남아프리카공화국"},
    "요하네스버그": {"대륙": "아프리카", "국가": "남아프리카공화국"},
    "카사블랑카": {"대륙": "아프리카", "국가": "모로코"},
    "마라케시": {"대륙": "아프리카", "국가": "모로코"},
    "나이로비": {"대륙": "아프리카", "국가": "케냐"},
    "다르에스살람": {"대륙": "아프리카", "국가": "탄자니아"},
    
    # 기타 (특별 지역)
    "몰디브": {"대륙": "아시아", "국가": "몰디브"},
    "괌": {"대륙": "오세아니아", "국가": "괌"},
    "사이판": {"대륙": "오세아니아", "국가": "사이판"},
    "제주도": {"대륙": "아시아", "국가": "한국"},
    "부산": {"대륙": "아시아", "국가": "한국"},
    "강릉": {"대륙": "아시아", "국가": "한국"},
    
    # 🟢 새로 추가된 태국 도시들
    "아유타야": {"대륙": "아시아", "국가": "태국"},
    "수코타이": {"대륙": "아시아", "국가": "태국"},
    "카오야이": {"대륙": "아시아", "국가": "태국"},
    "후아힌": {"대륙": "아시아", "국가": "태국"},
    "코사무이": {"대륙": "아시아", "국가": "태국"},
    "크라비": {"대륙": "아시아", "국가": "태국"}
}

def get_city_info(city_name):
    """도시명으로 대륙과 국가 정보를 가져오는 함수"""
    info = CITY_INFO.get(city_name, {"대륙": "기타", "국가": "기타"})
    return info["대륙"], info["국가"]

# 결과를 저장할 리스트 (🔧 매번 명확히 초기화!)
all_results = []
print("🔄 결과 저장소 초기화 완료")

#--- 크롬 드라이버 실행 ---#
import chromedriver_autoinstaller, random, os
chromedriver_autoinstaller.install()

import undetected_chromedriver as uc
from user_agents import parse

import selenium
print(selenium.__version__)

def make_user_agent(ua,is_mobile):
    user_agent = parse(ua)
    model = user_agent.device.model
    platform = user_agent.os.family
    platform_version = user_agent.os.version_string + ".0.0"
    version = user_agent.browser.version[0]
    ua_full_version = user_agent.browser.version_string
    architecture = "x86"
    print(platform)
    if is_mobile:
        platform_info = "Linux armv8l"
        architecture= ""
    else: # Window
        platform_info = "Win32"
        model = ""
    RET_USER_AGENT = {
        "appVersion" : ua.replace("Mozilla/", ""),
        "userAgent": ua,
        "platform" : f"{platform_info}",
        "acceptLanguage" : "ko-KR, kr, en-US, en",
        "userAgentMetadata":{
            "brands" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersionList" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersion":f"{ua_full_version}",
            "platform" :platform,
            "platformVersion":platform_version,
            "architecture":architecture,
            "model" : model,
            "mobile":is_mobile #True, False
        }
    }
    return RET_USER_AGENT

def generate_random_geolocation():
    ltop_lat = 37.75415601640249
    ltop_long = 126.86767642302573
    rbottom_lat = 37.593829172663945
    rbottom_long = 127.15276051439332

    targetLat = random.uniform(rbottom_lat, ltop_lat)
    targetLong = random.uniform(ltop_long,rbottom_long)
    return {"latitude":targetLat, "longitude" : targetLong, "accuracy":100}

# 🟢 파일명 안전 처리 함수 추가
def make_safe_filename(filename):
    """파일명에 사용할 수 없는 문자 제거"""
    unsafe_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|']
    safe_filename = filename
    for char in unsafe_chars:
        safe_filename = safe_filename.replace(char, '_')
    return safe_filename

options = uc.ChromeOptions()

UA = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Whale/4.32.315.22 Safari/537.36"
options.add_argument(f"--user-agent={UA}")
# User Agent 확인 : https://www.whatismybrowser.com/detect/what-is-my-user-agent/ 
# [중요] 위에 링크에서 나오는 값을 UA에 넣어줘야 함 

rand_user_folder = random.randrange(1,100)
raw_path = os.path.abspath("cookies")
try:
    shutil.rmtree(raw_path)
except:
    pass
os.makedirs(raw_path, exist_ok=True)
user_cookie_name = f"{raw_path}/{rand_user_folder}"
if os.path.exists(user_cookie_name) == False:
    os.makedirs(user_cookie_name, exist_ok=True)
# options.user_data_dir = user_cookie_name
# 쿠키 랜덤 설정

try:
    driver = uc.Chrome(user_data_dir=user_cookie_name, option = options)
except Exception as e:
    print('\n',"-"*50,"\n","-"*50,"\n")
    print("# 키홈 메세지 : 혹시 여기서 에러 발생시 [아래 블로그 참고 -> 재부팅 -> 다시 코드실행] 해보시길 바랍니다! \n (구글크롬 버젼 업그레이드 문제)")
    print('https://appfollow.tistory.com/102')  # 에러 메시지를 출력
    print('\n',"-"*50,"\n","-"*50,"\n")
    raise RuntimeError
    
UA_Data = make_user_agent(UA,False)
driver.execute_cdp_cmd("Network.setUserAgentOverride",UA_Data)
# # user agent data 설정

GEO_DATA = generate_random_geolocation()
driver.execute_cdp_cmd("Emulation.setGeolocationOverride", GEO_DATA)
driver.execute_cdp_cmd("Emulation.setUserAgentOverride", UA_Data)
driver.execute_cdp_cmd("Emulation.setNavigatorOverrides",{"platform":"Linux armv8l"})

# 썸네일 폴더 생성 (한 번만)
img_folder_path = os.path.abspath("") + "/myrealtripthumb_img"
try:
    shutil.rmtree(img_folder_path)
except:
    pass
os.makedirs(img_folder_path, exist_ok=True)
print("이미지 폴더 생성 완료")

# 🎉 여러 도시 크롤링 시작
print("="*60)
print(f"🌏 총 {len(CITIES_TO_SEARCH)}개 도시 검색 시작!")
print(f"검색할 도시들: {', '.join(CITIES_TO_SEARCH)}")
print(f"📊 현재 저장된 결과 수: {len(all_results)}개")  # 🔧 현재 상태 확인
print("="*60)

# 각 도시별로 반복 검색
for city_index, city_name in enumerate(CITIES_TO_SEARCH):
    print(f"\n🔍 [{city_index + 1}/{len(CITIES_TO_SEARCH)}] {city_name} 검색 시작...")
    
    try:
        # 🔧 수정: 대륙과 국가명을 맨 처음에 가져오기 (변수 정의 순서 중요!)
        continent, country = get_city_info(city_name)
        print(f"  🌍 대륙: {continent} | 국가: {country}")
        
        # 1. 메인 페이지로 이동 (매번 새로 시작)
        driver.get("https://www.myrealtrip.com/experiences/")
        time.sleep(random.uniform(3,8))
        print(f"  📱 마이리얼트립 페이지 열기 완료")
        
        # 2. 🟢 개선된 검색창 찾기 및 입력
        print(f"  🔍 '{city_name}' 검색창 찾는 중...")
        search_selectors = [
            (By.CSS_SELECTOR, "input[placeholder*='어디로']"),
            (By.CSS_SELECTOR, "input[type='text']"),
            (By.XPATH, "//input[contains(@placeholder, '어디로')]"),
            (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/input")  # 기존 방법도 백업으로 포함
        ]

        search_input = None
        for selector_type, selector_value in search_selectors:
            try:
                search_input = WebDriverWait(driver, WAIT_TIMEOUT).until(
                    EC.presence_of_element_located((selector_type, selector_value))
                )
                print(f"  ✅ 검색창을 찾았습니다!")
                break
            except TimeoutException:
                continue

        if not search_input:
            print(f"  ❌ {city_name}: 검색창을 찾을 수 없습니다. 다음 도시로...")
            continue

        # 검색어 입력
        search_input.clear()  # 이전 검색어 지우기
        search_input.send_keys(city_name)
        time.sleep(random.uniform(3,5))
        print(f"  📝 '{city_name}' 키워드 입력 완료")

        # 3. 🟢 개선된 검색 버튼 클릭
        print(f"  🔎 검색 버튼 찾는 중...")
        search_button_selectors = [
            (By.CSS_SELECTOR, "button[type='submit']"),
            (By.CSS_SELECTOR, ".search-btn"),
            (By.XPATH, "//button[contains(@class, 'search')]"),
            (By.XPATH, "//img[contains(@alt, '검색')]//parent::*"),
            (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/div/img")  # 기존 방법 백업
        ]

        search_clicked = False
        for selector_type, selector_value in search_button_selectors:
            try:
                search_button = WebDriverWait(driver, WAIT_TIMEOUT).until(
                    EC.element_to_be_clickable((selector_type, selector_value))
                )
                search_button.click()
                print(f"  ✅ 검색 버튼 클릭 성공!")
                search_clicked = True
                time.sleep(random.uniform(3,7))
                break
            except TimeoutException:
                continue

        if not search_clicked:
            print(f"  ❌ {city_name}: 검색 버튼을 찾을 수 없습니다. 다음 도시로...")
            continue

        # 4. 🟢 개선된 팝업 처리
        popup_selectors = [
            (By.CSS_SELECTOR, ".popup-close"),
            (By.CSS_SELECTOR, ".modal-close"),
            (By.XPATH, "//button[contains(@aria-label, '닫기')]"),
            (By.XPATH, "//button[contains(text(), '닫기')]"),
            (By.XPATH, "/html/body/div[15]/div[2]/button")  # 기존 방법도 포함
        ]

        popup_closed = False
        for selector_type, selector_value in popup_selectors:
            try:
                popup_button = WebDriverWait(driver, 5).until(  # 팝업은 짧은 시간만 기다림
                    EC.element_to_be_clickable((selector_type, selector_value))
                )
                popup_button.click()
                print(f"  ✅ 팝업창을 닫았습니다.")
                popup_closed = True
                time.sleep(random.uniform(1, 4))
                break
            except TimeoutException:
                continue

        if not popup_closed:
            print(f"  ℹ️ 팝업창이 없거나 이미 닫혀있습니다.")

        # 5. 🟢 개선된 전체 상품 보기 버튼 클릭
        print(f"  📋 전체 상품 보기 버튼 찾는 중...")
        view_all_selectors = [
            (By.XPATH, "//button[contains(text(), '전체')]"),
            (By.XPATH, "//span[contains(text(), '전체')]//parent::button"),
            (By.CSS_SELECTOR, "button[aria-label*='전체']"),
            (By.XPATH, "/html/body/div[4]/div[2]/div/div/div/span[21]/button")  # 기존 방법 백업
        ]

        view_all_clicked = False
        for selector_type, selector_value in view_all_selectors:
            try:
                view_all_button = WebDriverWait(driver, WAIT_TIMEOUT).until(
                    EC.element_to_be_clickable((selector_type, selector_value))
                )
                view_all_button.click()
                print(f"  ✅ 전체 상품 보기 클릭 성공!")
                view_all_clicked = True
                time.sleep(random.uniform(3,6))
                break
            except TimeoutException:
                continue

        if not view_all_clicked:
            print(f"  ⚠️ 전체 상품 보기 버튼을 찾을 수 없습니다. 현재 상품으로 진행...")

        # 6. 🟢 개선된 첫 번째 상품 클릭
        print(f"  🎯 첫 번째 상품 찾는 중...")
        first_product_selectors = [
            (By.CSS_SELECTOR, "a[href*='/experiences/'] img"),
            (By.CSS_SELECTOR, ".product-item:first-child img"),
            (By.XPATH, "//a[contains(@href, '/experiences/')][1]//img"),
            (By.XPATH, "/html/body/div[4]/div[2]/div/div/div[2]/main/div/div[3]/a[1]/div/div/img")  # 기존 방법
        ]

        product_clicked = False
        for selector_type, selector_value in first_product_selectors:
            try:
                first_product = WebDriverWait(driver, WAIT_TIMEOUT).until(
                    EC.element_to_be_clickable((selector_type, selector_value))
                )
                first_product.click()
                print(f"  ✅ 첫 번째 상품 클릭 성공!")
                product_clicked = True
                time.sleep(random.uniform(3,6))
                break
            except TimeoutException:
                continue

        if not product_clicked:
            print(f"  ❌ {city_name}: 상품을 찾을 수 없습니다. 다음 도시로...")
            continue

        # 7. 🟢 개선된 상품명 수집
        print(f"  📊 상품 정보 수집 중...")
        product_name = "정보 없음"
        title_selectors = [
            (By.CSS_SELECTOR, "h1"),
            (By.CSS_SELECTOR, ".product-title"),
            (By.XPATH, "//h1[contains(@class, 'title')]"),
            (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/h1")  # 기존 방법
        ]

        for selector_type, selector_value in title_selectors:
            try:
                title_element = WebDriverWait(driver, WAIT_TIMEOUT).until(
                    EC.presence_of_element_located((selector_type, selector_value))
                )
                product_name = title_element.text
                break
            except TimeoutException:
                continue

        time.sleep(random.uniform(3,5))

        # 8. 🟢 개선된 가격 정보 수집
        price = "정보 없음"
        price_selectors = [
            (By.CSS_SELECTOR, ".price"),
            (By.CSS_SELECTOR, "[class*='price']"),
            (By.XPATH, "//span[contains(text(), '원')]"),
            (By.XPATH, "/html/body/div[1]/main/div[1]/div[4]/div/div/div[2]/span[2]")  # 기존 방법
        ]

        for selector_type, selector_value in price_selectors:
            try:
                price_element = WebDriverWait(driver, WAIT_TIMEOUT).until(
                    EC.presence_of_element_located((selector_type, selector_value))
                )
                price = price_element.text
                break
            except TimeoutException:
                continue

        time.sleep(random.uniform(3,5))

        # 9. 🟢 개선된 평점 정보 수집 (중복 제거)
        grade_review = "정보 없음"
        rating_selectors = [
            (By.CSS_SELECTOR, ".rating"),
            (By.CSS_SELECTOR, "[class*='rating']"),
            (By.XPATH, "//span[contains(@class, 'rating')]"),
            (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/span/span[2]")  # 기존 방법
        ]

        for selector_type, selector_value in rating_selectors:
            try:
                rating_element = WebDriverWait(driver, WAIT_TIMEOUT).until(
                    EC.presence_of_element_located((selector_type, selector_value))
                )
                grade_review = rating_element.text
                break
            except TimeoutException:
                continue

        time.sleep(random.uniform(2,4))

        # 9-1. 🟢 리뷰 수 정보 수집 (새로 추가!)
        print(f"  📝 리뷰 수 정보 찾는 중...")
        review_count = ""  # 기본값은 빈 문자열
        review_count_selectors = [
            (By.XPATH, "//span[contains(text(), '리뷰')]"),
            (By.XPATH, "//span[contains(text(), 'review')]"),
            (By.XPATH, "//span[contains(text(), '후기')]"),
            (By.XPATH, "//span[contains(text(), '개')]//preceding-sibling::span"),
            (By.XPATH, "//span[contains(text(), '건')]"),
            (By.CSS_SELECTOR, "[class*='review']"),
            (By.CSS_SELECTOR, "[class*='count']"),
            (By.XPATH, "//div[contains(@class, 'review')]//span"),
            (By.XPATH, "//div[contains(@class, 'rating')]//span[contains(text(), '개')]"),
            (By.XPATH, "//span[contains(@class, 'rating')]//following-sibling::span"),
            # 숫자 + 리뷰/후기/개 패턴 찾기
            (By.XPATH, "//span[contains(text(), '개') and contains(text(), '리뷰')]"),
            (By.XPATH, "//span[contains(text(), '건') and contains(text(), '후기')]"),
            (By.XPATH, "//span[text()[contains(., '개') or contains(., '건') or contains(., 'review')]]"),
        ]

        for selector_type, selector_value in review_count_selectors:
            try:
                review_element = WebDriverWait(driver, 3).until(  # 리뷰수는 짧은 시간만 대기
                    EC.presence_of_element_located((selector_type, selector_value))
                )
                review_text = review_element.text.strip()
                
                # 리뷰/후기 관련 키워드와 숫자가 포함되어 있는지 확인
                review_keywords = ['리뷰', '후기', 'review', '개', '건']
                has_number = any(char.isdigit() for char in review_text)
                has_keyword = any(keyword in review_text.lower() for keyword in review_keywords)
                
                if has_number and has_keyword and len(review_text) < 50:  # 너무 긴 텍스트는 제외
                    review_count = review_text
                    print(f"  ✅ 리뷰 수 정보 발견: {review_count}")
                    break
                    
            except TimeoutException:
                continue

        if not review_count:
            print(f"  ℹ️ 리뷰 수 정보를 찾을 수 없습니다. (공란으로 처리)")

        time.sleep(random.uniform(3,5))

        # 10. 🟢 언어 정보 수집 (새로 추가!)
        print(f"  🌐 언어 정보 찾는 중...")
        language = ""  # 기본값은 빈 문자열
        language_selectors = [
            (By.XPATH, "//span[contains(text(), '언어')]//following-sibling::span"),
            (By.XPATH, "//div[contains(text(), '언어')]//following-sibling::div"),
            (By.CSS_SELECTOR, "[class*='language']"),
            (By.XPATH, "//span[contains(@class, 'language')]"),
            (By.XPATH, "//div[contains(@class, 'language')]"),
            (By.XPATH, "//span[contains(text(), '한국어')]"),
            (By.XPATH, "//span[contains(text(), '영어')]"),
            (By.XPATH, "//span[contains(text(), '중국어')]"),
            (By.XPATH, "//span[contains(text(), '일본어')]"),
            (By.XPATH, "//span[contains(text(), '태국어')]"),
            (By.XPATH, "//li[contains(text(), '언어')]"),
            (By.XPATH, "//p[contains(text(), '언어')]"),
            # 일반적인 언어 표기 패턴들
            (By.XPATH, "//span[contains(text(), 'Korean')]"),
            (By.XPATH, "//span[contains(text(), 'English')]"),
            (By.XPATH, "//span[contains(text(), 'Chinese')]"),
            (By.XPATH, "//span[contains(text(), 'Japanese')]"),
            (By.XPATH, "//span[contains(text(), 'Thai')]"),
        ]

        for selector_type, selector_value in language_selectors:
            try:
                language_element = WebDriverWait(driver, 3).until(  # 언어는 짧은 시간만 대기
                    EC.presence_of_element_located((selector_type, selector_value))
                )
                language_text = language_element.text.strip()
                
                # 언어 관련 키워드가 포함되어 있는지 확인
                language_keywords = ['언어', '한국어', '영어', '중국어', '일본어', '태국어', 
                                   'Korean', 'English', 'Chinese', 'Japanese', 'Thai',
                                   '중문', '일문', '한글', 'Language']
                
                if any(keyword in language_text for keyword in language_keywords):
                    language = language_text
                    print(f"  ✅ 언어 정보 발견: {language}")
                    break
                    
            except TimeoutException:
                continue

        if not language:
            print(f"  ℹ️ 언어 정보를 찾을 수 없습니다. (공란으로 처리)")

        time.sleep(random.uniform(1,3))

        # 11. 🟢 개선된 이미지 다운로드
        print(f"  🖼️ 상품 이미지 다운로드 중...")
        img_url = None
        image_selectors = [
            (By.CSS_SELECTOR, ".product-image img"),
            (By.CSS_SELECTOR, ".gallery img:first-child"),
            (By.XPATH, "//img[contains(@alt, '상품')]"),
            (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[3]/div/div/div/div[1]/div/div/div/div/img")  # 기존
        ]

        img_path = "이미지 없음"
        for selector_type, selector_value in image_selectors:
            try:
                img_element = WebDriverWait(driver, WAIT_TIMEOUT).until(
                    EC.presence_of_element_located((selector_type, selector_value))
                )
                img_url = img_element.get_attribute('src')
                break
            except TimeoutException:
                continue

        if img_url:
            try:
                safe_product_name = make_safe_filename(f"{city_name}_{product_name[:30]}")
                img_path = os.path.abspath("") + "/myrealtripthumb_img/" + safe_product_name + ".png"
                urllib.request.urlretrieve(img_url, img_path)
                print(f"  ✅ 이미지 다운로드 완료!")
            except Exception as e:
                print(f"  ⚠️ 이미지 다운로드 실패: {e}")
                img_path = "다운로드 실패"

        # 12. 현재 페이지 URL 가져오기
        current_url = driver.current_url

        # 13. 결과 저장 (대륙, 국가 정보는 이미 위에서 정의됨!)
        result = {
            '번호': city_index + 1,
            '대륙': continent,  # 🟢 이미 정의된 변수 사용!
            '국가': country,   # 🟢 이미 정의된 변수 사용!
            '도시': city_name,
            '상품명': product_name,
            '가격': price,
            '평점': grade_review,
            '리뷰수': review_count,
            '언어': language,
            '이미지_경로': img_path,
            'URL': current_url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S')
        }

        all_results.append(result)

        # 결과 출력
        print(f"  ✅ {city_name} 크롤링 완료!")
        print(f"     대륙: {continent} | 국가: {country}")  # 🟢 안전하게 사용!
        print(f"     상품명: {product_name[:40]}{'...' if len(product_name) > 40 else ''}")
        print(f"     가격: {price}")
        print(f"     평점: {grade_review}")
        print(f"     리뷰수: {review_count if review_count else '정보 없음'}")
        print(f"     언어: {language if language else '정보 없음'}")

        # 다음 검색을 위한 휴식 (마지막 도시가 아닌 경우)
        if city_index < len(CITIES_TO_SEARCH) - 1:
            wait_time = random.uniform(5, 10)
            print(f"  ⏰ 다음 검색까지 {wait_time:.1f}초 대기...")
            time.sleep(wait_time)

    except Exception as e:
        print(f"  ❌ {city_name} 처리 중 오류 발생: {e}")
        print(f"  📍 위치: {continent} > {country} > {city_name}")  # 🟢 이제 안전하게 사용 가능!
        print(f"  ➡️ 다음 도시로 계속 진행합니다...")
        continue

# 13. 최종 결과 출력 및 CSV 저장
print("\n" + "="*60)
print("🎉 모든 도시 검색 완료!")
print("="*60)

if all_results:
    print(f"\n📊 총 {len(all_results)}개 도시의 정보를 수집했습니다:")
    print("-" * 80)
    for result in all_results:
        print(f"{result['번호']}. {result['대륙']} > {result['국가']} > {result['도시']}")  # 🟢 대륙 > 국가 > 도시!
        print(f"   상품명: {result['상품명']}")
        print(f"   가격: {result['가격']}")
        print(f"   평점: {result['평점']}")
        print(f"   리뷰수: {result['리뷰수'] if result['리뷰수'] else '정보 없음'}")
        print(f"   언어: {result['언어'] if result['언어'] else '정보 없음'}")
        print(f"   URL: {result['URL']}")
        print("-" * 80)

    # 14. CSV 파일로 저장
    try:
        df = pd.DataFrame(all_results)
        timestamp = time.strftime('%Y%m%d_%H%M%S')
        filename = f"myrealtrip_여러도시_{timestamp}.csv"
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        
        print(f"\n💾 결과가 '{filename}' 파일로 저장되었습니다!")
        print(f"📁 파일 위치: {os.path.abspath(filename)}")
        print("💡 엑셀에서 바로 열어볼 수 있습니다!")
        
        # CSV 파일 내용 미리보기
        print(f"\n📋 CSV 파일 내용 미리보기:")
        print(df.to_string(index=False, max_colwidth=30))
        
    except Exception as e:
        print(f"\n❌ CSV 파일 저장 중 오류: {e}")
        print("수집된 데이터:")
        for result in all_results:
            print(result)

else:
    print("\n❌ 수집된 결과가 없습니다.")
    print("💡 도시명이나 인터넷 연결을 확인해보세요.")

print(f"\n🏁 크롤링 완료! 총 {len(all_results)}개 도시 정보 수집")
print("="*60)

🔄 결과 저장소 초기화 완료
4.25.0
Windows
이미지 폴더 생성 완료
🌏 총 1개 도시 검색 시작!
검색할 도시들: 방콕
📊 현재 저장된 결과 수: 0개

🔍 [1/1] 방콕 검색 시작...
  🌍 대륙: 아시아 | 국가: 태국
  📱 마이리얼트립 페이지 열기 완료
  🔍 '방콕' 검색창 찾는 중...
  ✅ 검색창을 찾았습니다!
  📝 '방콕' 키워드 입력 완료
  🔎 검색 버튼 찾는 중...
  ✅ 검색 버튼 클릭 성공!
  ✅ 팝업창을 닫았습니다.
  📋 전체 상품 보기 버튼 찾는 중...
  ✅ 전체 상품 보기 클릭 성공!
  🎯 첫 번째 상품 찾는 중...
  ✅ 첫 번째 상품 클릭 성공!
  📊 상품 정보 수집 중...
  📝 리뷰 수 정보 찾는 중...
  ✅ 리뷰 수 정보 발견: 후기 2,740개
  🌐 언어 정보 찾는 중...
  ✅ 언어 정보 발견: 한국어
  🖼️ 상품 이미지 다운로드 중...
  ✅ 이미지 다운로드 완료!
  ✅ 방콕 크롤링 완료!
     대륙: 아시아 | 국가: 태국
     상품명: 벨럭, 방콕 호텔→수완나품 공항/ 방콕 호텔간 짐 배송 서비스
     가격: 최대 17,000원 선착순 즉시 할인
     평점: 4.9 ·
     리뷰수: 후기 2,740개
     언어: 한국어

🎉 모든 도시 검색 완료!

📊 총 1개 도시의 정보를 수집했습니다:
--------------------------------------------------------------------------------
1. 아시아 > 태국 > 방콕
   상품명: 벨럭, 방콕 호텔→수완나품 공항/ 방콕 호텔간 짐 배송 서비스
   가격: 최대 17,000원 선착순 즉시 할인
   평점: 4.9 ·
   리뷰수: 후기 2,740개
   언어: 한국어
   URL: https://experiences.myrealtrip.com/products/3443042
----------------------------------------------